In [2]:
import pandas as pd
import sklearn
from matplotlib import pyplot
from datetime import datetime

In [19]:
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot

def parser(x):
    return datetime.strptime(x,'%Y-%m-%d')

series = read_csv("../data/IndFlaMin.csv", header=0, parse_dates=[1], squeeze=True, date_parser=parser).drop(columns=["Unnamed: 0"])
print(series)

           Date    Index  Inflation  FedMinWage
0    1913-01-01    9.800       0.00        0.00
1    1913-02-01    9.800       0.00        0.00
2    1913-03-01    9.800       0.00        0.00
3    1913-04-01    9.800       0.00        0.00
4    1913-05-01    9.700      -1.02        0.00
...         ...      ...        ...         ...
1208 2013-09-01  234.149       0.12        7.25
1209 2013-10-01  233.546      -0.26        7.25
1210 2013-11-01  233.069      -0.20        7.25
1211 2013-12-01  233.049      -0.01        7.25
1212 2014-01-01  233.916       0.37        7.25

[1213 rows x 4 columns]


In [20]:
# index => cpi index 
series["Year"] = series["Date"].dt.year
series["Month"] = series["Date"].dt.month


In [26]:
series = series.drop(columns="Date")

In [30]:
from statsmodels.tsa.stattools import adfuller

In [52]:
tr = adfuller(series["Index"])

In [50]:
def adfTest(values):
    tResult = adfuller(values)
    labels = ["ADF Test Statistic","p-value","#Lags Used","Number of Observations Used"]
    for value,label in zip(tResult, labels):
        print (label+ ":"+str(value))
    if tResult[1] <= 0.05:
        print ("strong evidence this is seasonal")
    else:
        print("This is not seasonal")

In [55]:
adfTest(series["FedMinWage"])

ADF Test Statistic:1.412152554300684
p-value:0.9971686557504474
#Lags Used:12
Number of Observations Used:1200
This is not seasonal


In [66]:
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(series.values.reshape(-1).tolist(), order=(3,1,1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

ValueError: endog is required to have ndim 1 but has ndim 2

In [25]:
import numpy as np
np.asarray(series)

array([[Timestamp('1913-01-01 00:00:00'), 9.8, 0.0, 0.0, 1913, 1],
       [Timestamp('1913-02-01 00:00:00'), 9.8, 0.0, 0.0, 1913, 2],
       [Timestamp('1913-03-01 00:00:00'), 9.8, 0.0, 0.0, 1913, 3],
       ...,
       [Timestamp('2013-11-01 00:00:00'), 233.069, -0.2, 7.25, 2013, 11],
       [Timestamp('2013-12-01 00:00:00'), 233.049, -0.01, 7.25, 2013, 12],
       [Timestamp('2014-01-01 00:00:00'), 233.916, 0.37, 7.25, 2014, 1]],
      dtype=object)

In [6]:
X = series[["Index","Inflation","FedMinWage"]]
y = series["Date"]
print(X.shape, y.shape)

(1213, 3) (1213,)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 33)

In [11]:
model = LinearRegression() 

In [12]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
model.score(X_test,y_test)

UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('float64')